Assignment 1. First ArcPy Lab

Tianzhe Wang


In [18]:
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.3\\ArcToolbox\\Scripts')
import arcpy 
from operator import itemgetter  # for sorting list of dictionaries

arcpy.env.overwriteOutput = True
from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"

arcpy.env.workspace = "C:\\Users\\wangtz\\Documents\\458\\Assgin1\\workspace"
cty = "C:\\Users\\wangtz\\Documents\\458\\Assgin1\\WashingtonFIPS.dbf"
bg = "C:\\Users\\wangtz\\Documents\\458\\Assgin1\\saep_bg10\\saep_bg10.shp"


The script below is for printing shapefile content.

In [19]:
# print Exists('saep_bg10.shp')
# des = Describe('C:\Users\wangtz\Documents\458\Assgin1\saep_bg10\saep_bg10.shp')
# print des

# see fields in shapefile
# t3 = "C:\\Users\\wangtz\\Downloads\\saep_bg10\\saep_bg10.shp"
# fields = arcpy.ListFields(t3)
# for field in fields:
#      print field.name
    
# bgrows = SearchCursor(bg)
# for row in bgrows:
#     print row.getValue("COUNTYFP10")
    
# cols = arcpy.ListFields(bg)
# for col in cols:
#      print col.name

# for row in ctyrows:
#     print ("CountyName: {}, FIPSCounty: {} ").format(
#         row.getValue("CountyName"),
#         row.getValue("FIPSCounty"),)

# for row in bgrows:
#     print row.getValue("COUNTYFP10")

Task 1
For each county in Washington, create a GeoJSON file whose filename is the county name and whose contents are the polygons for the block groups within that county.

To select block groups by counties, I used the approach of 'select by attribute', which is the arcpy.SelectLayerByAttribute_management() function. 

Within the shapefile of each county, I aggregated the population of 2016, and store the county name and population in dictionaries. Finally I stored the dictionaries in a list. 

In [20]:
# see fields in dbf
cty = "C:\Users\wangtz\\Documents\\458\\Assgin1\\WashingtonFIPS.dbf"
bg = "C:\\Users\\wangtz\\Documents\\458\\Assgin1\\saep_bg10\\saep_bg10.shp"

ctyrows = arcpy.SearchCursor(cty)
bgrows = arcpy.SearchCursor(bg)

population = [] 

# select block groups for each county
for county in ctyrows:
    county_fip = county.getValue("FIPSCounty")
    output_name = county.getValue("CountyName")   #shapefile name
    countyname = county.getValue("CountyName") + '01'  #layer
    arcpy.MakeFeatureLayer_management(bg, countyname)      
    # print county_fip
    f2 = arcpy.AddFieldDelimiters(countyname, "COUNTYFP10") # to make the string fit for query
    query = "{} = '{}'".format(f2, county_fip)
    # print query
    arcpy.SelectLayerByAttribute_management(countyname, "NEW_SELECTION", query)
    arcpy.CopyFeatures_management(countyname, "{}.shp".format(output_name))
    
    call(['C:\\OSGeo4W64\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927',
      'C:\\Users\\wangtz\\Documents\\458\\Assgin1\\test\\{}.geojson'.format(output_name),
      'C:\\Users\\wangtz\\Documents\\458\\Assgin1\\workspace\\{}.shp'.format(output_name)]) # in seperate folder
    
    # county population
    pop = 0
    bg2 = 'C:\\Users\\wangtz\\Documents\\458\\Assgin1\\workspace\\{}.shp'.format(output_name)
    bg2rows = arcpy.SearchCursor(bg2)
    for bg_single in bg2rows:
        pop += bg_single.getValue("POP2016")
    d= {'county population':int(pop),'county name':output_name} # each dictionary has the county name and county population
    population.append(d)


Task 2
Prints out a ranked list, in descending order, of the ten largest total populations in 2016 of counties in Washington, according to this dataset.


In [21]:
# create a sorted list in descending order, based on the population value
temp = sorted(population, key=itemgetter('county population'), reverse=True)

# print out 10 largest total populations in 2016 of counties in Washington
for i in temp[0:10:1]:
    print '{} County, population: {}'.format(i['county name'],i['county population'])

King County, population: 2105099
Pierce County, population: 844489
Snohomish County, population: 772860
Spokane County, population: 492530
Clark County, population: 461010
Thurston County, population: 272690
Kitsap County, population: 262589
Yakima County, population: 250899
Whatcom County, population: 212540
Benton County, population: 190500


Notes:
1. I used ArcMap 10.3 from department lab, it turns out that arcpy.da.SearchCursor cannot be executed on lab machines. So I still use arcpy.SearchCursor in the file.

2. When converting coordinate system of each county, I found all of them are in "1983_HARN_StatePlane_Washington_South_FIPS_4602_Feet". It might not be so precise for counties in north washington as those in south of state. 